In [35]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

# computation graph
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

# session
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

# auto session
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)
    
# run all variables
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)
    
# interactive session
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

# add node to default graph
x1 = tf.Variable(1, name="x1")
print(x1.graph is tf.get_default_graph())

# add node to new graph
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2, name="x2")
    print(x2.graph is tf.get_default_graph())
print(x2.graph is tf.get_default_graph())
print(x2.graph is graph)
    
# reset graph
tf.reset_default_graph()

# evaluate twice
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

# evaluate once
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y.eval())
    print(z.eval())
    
# linear regression with tensor flow
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing["target"].reshape(-1,1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)
tf.reset_default_graph()
    
# batch gradient descent with tensor flow
scaler = StandardScaler()
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)
n_epochs = 1000
learning_rate = 0.01
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing["target"].reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2./m * tf.matmul(tf.transpose(X), error)
#gradients = tf.gradients(mse, [theta])[0]
#training_op = tf.assign(theta, theta - learning_rate * gradients)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
    print(best_theta)
tf.reset_default_graph()  

# place holders
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})
print(B_val_1)
print(B_val_2)

# mini-batch gradient descent
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
batch_size = 100
n_batches = int(np.ceil(m / batch_size))
def fetch_batch(epoch, batch_index, batch_size):
    global scaled_housing_data_plus_bias
    global housing
    X_batch = scaled_housing_data_plus_bias[batch_size*batch_index:
                                            batch_size*(batch_index+1),:]
    y_batch = housing["target"].reshape(-1,1)[batch_size*batch_index:
                                            batch_size*(batch_index+1),:]
    return X_batch, y_batch
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
#        if epoch % 100 == 0:
#            print("Epoch", epoch, "MSE =", mse.eval())
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
    print(best_theta)

42
42
42
42
True
True
False
True
10
15
10
15
[[ -3.71851807e+01]
 [  4.36337471e-01]
 [  9.39523336e-03]
 [ -1.07113101e-01]
 [  6.44792199e-01]
 [ -4.03380000e-06]
 [ -3.78137082e-03]
 [ -4.23484027e-01]
 [ -4.37219113e-01]]
Epoch 0 MSE = 7.35761
Epoch 100 MSE = 4.96353
Epoch 200 MSE = 4.90784
Epoch 300 MSE = 4.8801
Epoch 400 MSE = 4.86009
Epoch 500 MSE = 4.84543
Epoch 600 MSE = 4.83466
Epoch 700 MSE = 4.82673
Epoch 800 MSE = 4.82087
Epoch 900 MSE = 4.81653
[[ 0.51642299]
 [ 0.84923303]
 [ 0.15414011]
 [-0.24327306]
 [ 0.26121089]
 [ 0.00811641]
 [-0.04273812]
 [-0.62349892]
 [-0.59356761]]
[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]
[[  5.18668890e-01]
 [  9.95533288e-01]
 [  9.72433686e-02]
 [ -3.68002862e-01]
 [  3.56470525e-01]
 [ -3.56882904e-03]
 [  9.67795379e-04]
 [ -2.38164261e-01]
 [ -1.48855281e+00]]
